In [ ]:
# Problem 1:  Some of the facilities charge a fee to members, but some do not.
# Please list the names of the facilities that do. 

SELECT * FROM `Facilities` WHERE membercost = 0

# Badminton Court
# Table Tennis
# Snooker Table
# Pool Table

In [ ]:
# Problem 2: How many facilities do not charge a fee to members? 

SELECT COUNT(*) FROM `Facilities` WHERE membercost = 0

# 4

In [ ]:
# Problem 3 How can you produce a list of facilities that charge a fee to members,
# where the fee is less than 20% of the facility's monthly maintenance cost?
# Return the facid, facility name, member cost, and monthly maintenance of the
# facilities in question.

SELECT facid,
	   name,
       membercost, 
       monthlymaintenance
FROM `Facilities` 
WHERE membercost < .2 * monthlymaintenance


#facid	name	      membercost	monthlymaintenance	
#0	  Tennis Court 1	5.0	            200
#1	  Tennis Court 2	5.0	            200
#2	  Badminton Court	0.0	            50
#3	  Table Tennis	    0.0	            10
#4	  Massage Room 1	9.9	            3000
#5	  Massage Room 2	9.9	            3000
#6	  Squash Court	    3.5	            80
#7	  Snooker Table	    0.0          	15
#8	  Pool Table	    0.0         	15

# Seems like it is all of them.

In [ ]:
# Problem 4 How can you retrieve the details of facilities with ID 1 and 5? 
# Write the query without using the OR operator.

SELECT * FROM `Facilities` WHERE name LIKE '%2'

In [ ]:
# Problem 5 How can you produce a list of facilities, with each labelled as
# 'cheap' or 'expensive', depending on if their monthly maintenance cost is
# more than $100? Return the name and monthly maintenance of the facilities
# in question.


SELECT name,
       monthlymaintenance,
       CASE WHEN monthlymaintenance > 100 THEN 'Expensive'
            ELSE 'Cheap' END AS Price_Point
    FROM `Facilities`
    
#name	       monthlymaintenance	Price_Point	
#Tennis Court 1	  200	             Expensive
#Tennis Court 2	  200            	 Expensive
#Badminton Court  50	             Cheap
#Table Tennis	  10	             Cheap
#Massage Room 1	  3000	             Expensive
#Massage Room 2	  3000	             Expensive
#Squash Court	  80	             Cheap
#Snooker Table	  15	             Cheap
#Pool Table	      15	             Cheap

In [ ]:
# Problem 6 You'd like to get the first and last name of the last member(s)
# who signed up. Do not use the LIMIT clause for your solution.

SELECT firstname, surname, joindate FROM `Members` ORDER BY joindate DESC

# Probably a way to shorten this...

In [ ]:
# Problem 7 How can you produce a list of all members who have used a tennis court?
# Include in your output the name of the court, and the name of the member
# formatted as a single column. Ensure no duplicate data, and order by
# the member name.

SELECT
  CONCAT(firstname, " ", surname) AS member_fullname,
  F.name as court_name
FROM
  `Bookings` B
JOIN
  `Facilities` F
ON
  B.facid = F.facid
JOIN
  `Members` M
ON
  M.memid = B.memid
WHERE
  F.name LIKE 'Tennis Court%'
GROUP BY 1,2
ORDER BY 1

In [ ]:
# Problem 8 How can you produce a list of bookings on the day of 2012-09-14 which 
# will cost the member (or guest) more than $30? Remember that guests have
# different costs to members (the listed costs are per half-hour 'slot'), and
# the guest user's ID is always 0. Include in your output the name of the
# facility, the name of the member formatted as a single column, and the cost.
# Order by descending cost, and do not use any subqueries.

SELECT facilities.name, 
	   concat(members.firstname, '_', members.surname) as member_name,
	   CASE WHEN members.memid = 0 THEN facilities.guestcost*bookings.slots
		ELSE facilities.membercost*bookings.slots END AS costs
FROM `Bookings` bookings
JOIN `Facilities` facilities ON bookings.facid = facilities.facid
JOIN `Members` members ON bookings.memid = members.memid
WHERE bookings.starttime LIKE '2012-09-14%'
HAVING costs > 30
ORDER BY costs DESC

In [ ]:
# Problem 9 This time, produce the same result as in Q8, but using a subquery.

SELECT facilities.name, 
	   concat(members.firstname, '_', members.surname) as member_name,
	   CASE WHEN members.memid = 0 THEN facilities.guestcost*bookings.slots
		ELSE facilities.membercost*bookings.slots END AS costs
FROM (
	SELECT memid, facid, slots FROM `Bookings` WHERE Bookings.starttime LIKE '2012-09-14%'
	 ) bookings
JOIN `Facilities` facilities ON bookings.facid = facilities.facid
JOIN `Members` members ON bookings.memid = members.memid
HAVING costs > 30
ORDER BY costs DESC

In [ ]:
# Problem 10 Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members!

SELECT facilities.name as facility_name,
	   SUM(CASE WHEN members.memid = 0 THEN facilities.guestcost*bookings.slots 
		ELSE facilities.membercost*bookings.slots END) as revenue
FROM `Bookings` bookings
JOIN `Facilities` facilities ON bookings.facid = facilities.facid
JOIN `Members` members ON bookings.memid = members.memid
GROUP BY facilities.name
HAVING revenue < 1000
ORDER BY revenue